In [5]:
#from esm_likelihood import get_esm_likelihood_scores
#from Bio import SeqIO
import pandas as pd

#testset_file = '../datasets/InterProUniprotPF03272prepared_test.fasta'
#testset = list(SeqIO.parse(testset_file, 'fasta'))

# filter sequences with more than 1024 amino acids because ESM can't handle longer sequences
#testset = [seq for seq in testset if len(seq) <= 1024]

# Calculate cutoff (10th percentile of ESM likelihood scores for the test set)
#testset_df = get_esm_likelihood_scores(testset)
testset_df = pd.read_csv("intermediate/basemodel-RITA_l-generated.fasta_testset_scores.csv")
cutoff = testset_df['score'].quantile(q=0.1)
cutoff

-0.4302726149559021

In [6]:
import glob

results = {}

for file in glob.glob("intermediate/*fasta_scores.csv"):
    df = pd.read_csv(file)
    filtered = df[df['score'] > cutoff]
    ids = list(filtered['id'])
    results[file] = ids

In [7]:
for k,v in results.items():
    print(k,len(v))

intermediate/finetuned-RITA_s-generated.fasta_scores.csv 20
intermediate/prompt-tuning-clustered-100-RITA_xl-fromvocab-True-seed-1-generated.fasta_scores.csv 18
intermediate/prompt-tuning-clustered-100-RITA_s-fromvocab-True-seed-0-generated.fasta_scores.csv 4
intermediate/prompt-tuning-clustered-100-RITA_l-fromvocab-True-seed-1-generated.fasta_scores.csv 8
intermediate/prompt-tuning-clustered-100-RITA_m-fromvocab-True-seed-2-generated.fasta_scores.csv 7
intermediate/prompt-tuning-clustered-100-RITA_l-fromvocab-True-seed-2-generated.fasta_scores.csv 10
intermediate/prompt-tuning-clustered-100-RITA_xl-fromvocab-True-seed-0-generated.fasta_scores.csv 18
intermediate/prompt-tuning-clustered-100-RITA_m-fromvocab-True-seed-1-generated.fasta_scores.csv 6
intermediate/finetuned-RITA_xl-generated.fasta_scores.csv 81
intermediate/prompt-tuning-clustered-100-RITA_xl-fromvocab-True-seed-2-generated.fasta_scores.csv 19
intermediate/basemodel-RITA_m-generated.fasta_scores.csv 11
intermediate/prompt-

In [8]:
import re

num_seqences_per_file = 193

pattern = re.compile(".*(RITA_\w{1,2})-.*")
getModel = lambda filename: pattern.match(filename).groups()[0]

data = [
    (getModel(k), 'pt', len(v)*100/num_seqences_per_file) if 'prompt-tuning' in k 
    else ((getModel(k),'base',len(v)*100/num_seqences_per_file)) if 'basemodel' in k
    else ((getModel(k),'fine',len(v)*100/num_seqences_per_file))
    for k,v in results.items()
]

order = ['RITA_s', 'RITA_m', 'RITA_l', 'RITA_xl']
data = sorted(data, key=lambda item: order.index(item[0]))
data

[('RITA_s', 'fine', 10.362694300518134),
 ('RITA_s', 'pt', 2.0725388601036268),
 ('RITA_s', 'pt', 3.1088082901554404),
 ('RITA_s', 'base', 2.5906735751295336),
 ('RITA_s', 'pt', 2.5906735751295336),
 ('RITA_m', 'pt', 3.626943005181347),
 ('RITA_m', 'pt', 3.1088082901554404),
 ('RITA_m', 'base', 5.699481865284974),
 ('RITA_m', 'pt', 2.5906735751295336),
 ('RITA_m', 'fine', 17.098445595854923),
 ('RITA_l', 'pt', 4.1450777202072535),
 ('RITA_l', 'pt', 5.181347150259067),
 ('RITA_l', 'pt', 4.1450777202072535),
 ('RITA_l', 'fine', 31.088082901554404),
 ('RITA_l', 'base', 7.772020725388601),
 ('RITA_xl', 'pt', 9.32642487046632),
 ('RITA_xl', 'pt', 9.32642487046632),
 ('RITA_xl', 'fine', 41.968911917098445),
 ('RITA_xl', 'pt', 9.844559585492227),
 ('RITA_xl', 'base', 5.181347150259067)]

In [ ]:
pd.DataFrame(data, columns=['model', 'method', 'percentage']).to_csv('activity.csv', index=False)